## ONBORADING DAY 3 SOLUTIONS
by Giovani Goltara

# Summary

- **Brooklyn** has the highest number of schools in New York.
- **English Language Learning** data is available only for **Manhattan**, although other boroughs appear to have more foreign/migrant students.
- **Special Education** data is also only available for **Manhattan**.

# Setup

In [38]:
import pandas as pd
import psycopg2 

In [39]:
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

In [40]:
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

/var/folders/nc/rtwbjn4545v3r4g5t0kbb1hh0000gn/T/ipykernel_52249/1731849884.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


# Task 01
How many schools are there per Borough?

In [41]:
query = 'SELECT borough, COUNT(*) FROM nyc_schools.high_school_directory GROUP BY borough;'
df = pd.read_sql(query, conn)
df

/var/folders/nc/rtwbjn4545v3r4g5t0kbb1hh0000gn/T/ipykernel_52249/2924021730.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,borough,count
0,Brooklyn,121
1,Queens,80
2,Staten Island,10
3,Manhattan,106
4,Bronx,118


# Task 2
What is the average % of English Language Learners (ELL) per borough?

In [42]:
query = """
SELECT 
	dir.borough,
	avg(dem.ell_percent)
FROM
	nyc_schools.high_school_directory as dir
	left join nyc_schools.school_demographics as dem
	on dir.dbn = dem.dbn
GROUP BY
	dir.borough 
    """
df = pd.read_sql(query, conn)
df

/var/folders/nc/rtwbjn4545v3r4g5t0kbb1hh0000gn/T/ipykernel_52249/1401796513.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,borough,avg
0,Brooklyn,NaN
1,Queens,NaN
2,Staten Island,NaN
3,Manhattan,7.5725
4,Bronx,NaN


# Task 3
Top 3 schools in each borough with the highest percentage of special education students

In [43]:
query = """
WITH ranking AS (
	SELECT 
		dir.borough,
		ROW_NUMBER() OVER (PARTITION BY dir.borough ORDER BY dem.sped_percent desc) AS sped_rank,
		dir.dbn,
		dir.school_name,
        dem.sped_percent
	FROM
	nyc_schools.high_school_directory AS dir
	join nyc_schools.school_demographics AS dem
	ON dir.dbn = dem.dbn
)
SELECT *
FROM ranking 
WHERE sped_rank <= 3
"""
df = pd.read_sql(query, conn)
df

/var/folders/nc/rtwbjn4545v3r4g5t0kbb1hh0000gn/T/ipykernel_52249/2534042740.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,borough,sped_rank,dbn,school_name,sped_percent
0,Manhattan,1,01M450,East Side Community School,28.8
1,Manhattan,2,01M450,East Side Community School,27.7
2,Manhattan,3,01M450,East Side Community School,26.7
